In [ ]:
import pandas as pd
import os
from tqdm import tqdm
import gc
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Construct relative path
finance_news = os.path.join(".", "FINANCIAL NEWS", "finance_data.csv")

# Minimal file path check
if not os.path.exists(finance_news):
    print(f"File not found at: {os.path.abspath(finance_news)}")
    print("Current working directory:", os.getcwd())
    print("Directory contents:", os.listdir(os.getcwd()))
    exit()

# Read the CSV file
df = pd.read_csv(finance_news)

# Verify row count
initial_rows = len(df)
print(f"Initial row count: {initial_rows}")

# Verify and process columns
required_columns = ['Date', 'Article_title', 'Stock_symbol', 'Article']
if not all(col in df.columns for col in required_columns):
    print("Error: Missing required columns. Available columns:", list(df.columns))
    exit()

# Keep only required columns
df = df[['Date', 'Article_title', 'Stock_symbol', 'Article']]

df['text'] = df['Article_title'].astype(str) + ' ' + df['Article'].fillna('').astype(str)
# Create df1 and drop original columns
df1 = df[['Date', 'Stock_symbol', 'text']].copy()

# Clear df to free memory
del df
gc.collect()

/home/jonye/anaconda3/envs/llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


/tmp/ipykernel_167796/1774218465.py:26: DtypeWarning: Columns (2,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(finance_news)


Initial row count: 13057514


160

In [ ]:
# Load FinBERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

# Move model to GPU
model = model.to(device)
model.eval()

# Custom Dataset for lazy tokenization
class TextDataset(Dataset):
    def __init__(self, series, max_length=512):
        self.series = series  # Store Pandas Series directly
        self.max_length = max_length

    def __len__(self):
        return len(self.series)

    def __getitem__(self, idx):
        text = self.series.iloc[idx]
        text = "" if pd.isna(text) or str(text).strip() == "" else str(text)
        return {'text': text, 'is_empty': text.strip() == ""}

# Batch inference function with mixed precision and chunking
def batch_sentiment_scores(series, tokenizer, batch_size=32, chunk_size=100000):
    scores = []
    sentiments = []
    labels = ['positive', 'negative', 'neutral']
    is_cuda = torch.cuda.is_available()

    # Process in chunks
    for start in tqdm(range(0, len(series), chunk_size), desc="Processing chunks"):
        end = min(start + chunk_size, len(series))
        chunk_series = series[start:end]

        # Create dataset and dataloader for chunk
        dataset = TextDataset(chunk_series, max_length=512)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

        chunk_scores = []
        chunk_sentiments = []

        with torch.no_grad():
            for batch in tqdm(dataloader, desc=f"Processing batches (chunk {start//chunk_size + 1})", total=len(dataloader)):
                texts = batch['text']
                is_empty = batch['is_empty']

                # Tokenize the batch with padding and truncation
                inputs = tokenizer(
                    texts,
                    return_tensors="pt",
                    max_length=512,
                    truncation=True,
                    padding='longest'  # Pad to the longest sequence in the batch
                )

                input_ids = inputs['input_ids'].to(device)
                attention_mask = inputs['attention_mask'].to(device)

                # Use mixed precision if CUDA is available
                if is_cuda:
                    with torch.amp.autocast(device_type='cuda'):
                        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                else:
                    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                print(f"output of the model {outputs}")
                probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
                probs = probs.cpu().numpy()
                batch_scores = np.max(probs, axis=1)
                batch_sentiments = [labels[np.argmax(p)] for p in probs]

                # Handle empty texts in batch
                for i, empty in enumerate(is_empty):
                    if empty:
                        batch_scores[i] = np.nan
                        batch_sentiments[i] = "neutral"

                chunk_scores.extend(batch_scores)
                chunk_sentiments.extend(batch_sentiments)

                # Clean up
                if is_cuda:
                    torch.cuda.empty_cache()

        scores.extend(chunk_scores)
        sentiments.extend(chunk_sentiments)

        # Clear chunk memory
        del chunk_series, dataset, dataloader
        gc.collect()

    return scores, sentiments

# Apply batched sentiment analysis
scores, sentiments = batch_sentiment_scores(df1['text'], tokenizer, batch_size=1024, chunk_size=100000)

# Assign results with efficient dtypes
df1['sentiment_score'] = pd.Series(scores, dtype="float32")
df1['sentiment'] = pd.Series(sentiments, dtype="category")

# Verify final row count
print(f"Final row count: {len(df1)}")
if len(df1) != initial_rows:
    print(f"Error: Rows were dropped during processing. Initial rows: {initial_rows}, Final rows: {len(df1)}")

# Display results
print("\nDataFrame with sentiment scores:")
print(df1.head())
print("\nNumber of missing sentiment scores:", df1['sentiment_score'].isna().sum())

# Save the DataFrame
output_file = os.path.join(".", "FINANCIAL NEWS", "finance_data_with_sentiment.csv")
df1.to_csv(output_file, index=False, compression=None)
print(f"DataFrame saved to: {output_file}")

# Clear df1 to free memory
del df1
gc.collect()
df2 = pd.read_csv("./FINANCIAL NEWS/finance_data_with_sentiment.csv")
df3 = df2.drop("text", axis=1).copy()
# Convert sentiment to numeric values
sentiment_mapping = {'positive': 1, 'neutral': 0, 'negative': -1}
df3['sentiment_numeric'] = df3['sentiment'].map(sentiment_mapping)

# Calculate adjusted sentiment score (sentiment_numeric * sentiment_score)
df3['adjusted_sentiment_score'] = df3['sentiment_numeric'] * df3['sentiment_score']

# Convert date column to date-only (removing time)
df3['date_only'] = pd.to_datetime(df3['Date']).dt.date

df3 = df3[["date_only","Stock_symbol","adjusted_sentiment_score"]]
df3 = df3.groupby(['Stock_symbol', 'date_only']).agg({
    'adjusted_sentiment_score': 'mean',
    # Add other columns here with appropriate aggregations, e.g., 'other_column': 'first'
}).reset_index()
distinct_values = df3['Stock_symbol'].unique()
len(distinct_values)
df3.to_csv('output.csv', index=False)
df3 = pd.read_csv('output.csv')
sp500_tickers = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "andrewmvd/sp-500-stocks",
    "sp500_companies.csv",
)
# Assuming df3 and sp500_tickers are your DataFrames
merged_df = pd.merge(df3, sp500_tickers, how='inner', left_on='Stock_symbol', right_on='Symbol')
merged_df = merged_df.drop(columns=['Symbol'], errors='ignore')
merged_df = merged_df.drop(columns=['Symbol'], errors='ignore')

print("Merged DataFrame:")
print(merged_df)

Processing chunks:   0%|          | 0/131 [00:00<?, ?it/s]

output of the model SequenceClassifierOutput(loss=None, logits=tensor([[-0.0068, -0.9526,  1.2002],
        [ 0.1642, -1.1592,  1.1045],
        [-0.5542, -0.4067,  1.5811],
        ...,
        [ 1.8311, -2.1484, -0.8320],
        [ 1.9365, -2.2324, -1.1055],
        [-0.8936, -1.0674,  2.5117]], device='cuda:0', dtype=torch.float16), hidden_states=None, attentions=None)


output of the model SequenceClassifierOutput(loss=None, logits=tensor([[-0.4700, -1.2344,  1.8809],
        [ 2.0137, -2.2109, -1.0596],
        [-1.2939, -0.2571,  2.3633],
        ...,
        [-1.5635,  1.8340,  0.6035],
        [-0.5322, -1.7080,  2.4824],
        [ 1.4062, -1.4688, -0.0137]], device='cuda:0', dtype=torch.float16), hidden_states=None, attentions=None)


output of the model SequenceClassifierOutput(loss=None, logits=tensor([[-0.8091, -0.0350,  1.2676],
        [-1.4023,  2.1719,  0.0705],
        [-1.0625,  0.0509,  1.9209],
        ...,
        [-0.3035, -1.9092,  2.4355],
        [-1.0732,  2.1719, -1.0615],
        [-1.3369,  2.3750, -0.8403]], device='cuda:0', dtype=torch.float16), hidden_states=None, attentions=None)


output of the model SequenceClassifierOutput(loss=None, logits=tensor([[-1.3701,  2.5977, -1.0840],
        [ 1.9854, -2.2676, -1.0547],
        [-0.6753, -1.0645,  2.4434],
        ...,
        [-1.3018, -0.3940,  2.4648],
        [-1.3320,  0.2744,  1.8682],
        [ 0.5366, -1.6924,  1.0430]], device='cuda:0', dtype=torch.float16), hidden_states=None, attentions=None)


...

output of the model SequenceClassifierOutput(loss=None, logits=tensor([[ 0.2637, -1.8223,  1.7227],
        [ 0.0543, -1.7959,  1.8965],
        [ 0.1548, -1.6455,  1.6914],
        ...,
        [ 0.3105, -1.8232,  1.6670],
        [ 0.2964, -1.6836,  1.5361],
        [ 0.1550, -1.5850,  1.6387]], device='cuda:0', dtype=torch.float16), hidden_states=None, attentions=None)


output of the model SequenceClassifierOutput(loss=None, logits=tensor([[ 0.1715, -1.6895,  1.6699],
        [ 0.0859, -1.6094,  1.7979],
        [ 0.1426, -1.7090,  1.7393],
        ...,
        [ 0.0839, -1.6455,  1.8320],
        [ 0.2595, -1.7158,  1.6387],
        [ 0.1121, -1.5283,  1.6240]], device='cuda:0', dtype=torch.float16), hidden_states=None, attentions=None)


output of the model SequenceClassifierOutput(loss=None, logits=tensor([[ 0.1642, -1.6514,  1.6641],
        [ 0.0185, -1.5312,  1.7412],
        [ 0.1475, -1.7354,  1.7002],
        ...,
        [-0.0918, -1.5879,  1.9219],
        [ 0.2383, -1.8604,  1.7822],
        [-0.0029, -1.6240,  1.8906]], device='cuda:0', dtype=torch.float16), hidden_states=None, attentions=None)


Processing chunks:  11%|█         | 14/131 [10:51<1:30:42, 46.52s/it]


KeyboardInterrupt: 